[[Python] Keras-RLで簡単に強化学習(DQN)を試す](http://qiita.com/inoory/items/e63ade6f21766c7c2393)を参考に、エージェントを作成する。FXの自動取引を行い、利益を出すのが目標。

In [ ]:
import numpy as np
import pandas as pd
import talib
from logging import getLogger, StreamHandler, DEBUG, INFO

from hist_data import HistData
from fx_trade import FXTrade
from deep_fx import DeepFX

Using TensorFlow backend.


In [ ]:
logger = getLogger(__name__)
handler = StreamHandler()
handler.setLevel(INFO)
logger.setLevel(INFO)
logger.addHandler(handler)

In [ ]:
#import imp
#import sys
#del(hist_data)
#from hist_data import HistData
#del(hist_data)
#imp.reload(hist_data)
#imp.reload(sys.modules[hist_data.__module__])
hd = HistData(csv_path = 'historical_data/USDJPY.hst_.csv',
                     begin_date='2010-09-01T00:00:00',
                     end_date='2010-09-07T23:59:59')

In [ ]:
hd.data()
#len(hist_data.data())

,Open,High,Low,Close,Volatility
Date,,,,,
2010-09-01 00:00:00,84.17,84.18,84.14,84.18,13.0
2010-09-01 00:01:00,84.18,84.18,84.14,84.18,11.0
2010-09-01 00:02:00,84.18,84.18,84.14,84.18,14.0
2010-09-01 00:03:00,84.18,84.18,84.15,84.18,8.0
2010-09-01 00:04:00,84.17,84.17,84.14,84.17,13.0
2010-09-01 00:05:00,84.17,84.17,84.14,84.17,6.0
2010-09-01 00:06:00,84.17,84.17,84.12,84.14,18.0
2010-09-01 00:07:00,84.14,84.14,84.12,84.14,13.0
2010-09-01 00:08:00,84.14,84.15,84.12,84.15,9.0


In [ ]:
env = FXTrade(1000000, 0.08, hd, logger=logger)
#env = FXTrade(1000000, 0.08, h, logger=logger)
prepared_model_filename = None #'Keras-RL_DQN_FX_model_meanq1.440944e+06_episode00003.h5'
dfx = DeepFX(env, 'test', prepared_model_filename=prepared_model_filename)

In [ ]:
#import imp
#import sys
#import deep_fx
#from deep_fx import DeepFX
##del(hist_data)
##from deep
##del(hist_data)
#imp.reload(deep_fx)
##imp.reload(sys.modules[deep_fx.__module__])
##reload(deep_fx)

In [ ]:
is_to_train = True
if is_to_train:
    dfx.train(is_for_time_measurement=True)
else:
    dfx.test(1, [EpisodeLogger()])

2010-09-01 00:00:00 84.180000


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 2)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 9         
_________________________________________________________________
activation_1 (Activation)    (None, 3)                 0         
Total params: 9
Trainable params: 9
Non-trainable params: 0
_________________________________________________________________
None
17/10/30 00:12:10
Training for 14390 steps ...
Training for 14390 steps ...
_reset START
self._seed: 100000
Start datetime: 2010-09-01T00:00:00.000000000
_reset END
now_datetime: 2010-09-01 00:01:00
len(self.hist_data.data()) - 1: 7140
now_datetime: 2010-09-01 00:02:00
len(self.hist_data.data()) - 1: 7140
now_datetime: 2010-09-01 00:03:00
len(self.hist_data.data()) - 1: 7140
now_datetime: 2010-09-01 00:0

2010-09-02 00:00:00 84.420000


In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
data = hist_data.data()['Close']
x = data.index
y = data.values
sd = 1
upper, middle, lower = talib.BBANDS(data.values, timeperiod=20, matype=talib.MA_Type.SMA, nbdevup=sd, nbdevdn=sd)
[plt.plot(x, val) for val in [y, upper, middle, lower]]

In [ ]:
data.values

## References

- [http://recruit.gmo.jp/engineer/jisedai/blog/deep-q-learning/](Deep Q-LearningでFXしてみた)